In [1]:
import pandas as pd
import numpy as np

In [2]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

In [3]:
from datetime import datetime, timedelta

## 유가 데이터

In [4]:
df = pd.read_csv('./data/daum_oil.csv')
df_dubai = df[df['company'] == '두바이유선물']
df_dubai['date'] = pd.to_datetime(df_dubai['date'])
df_d = df_dubai.set_index('date').loc['2017-2-10':'2022-02-10'].resample('W-MON',label = 'left', closed='right')['종가'].last()
df_d = df_d.reset_index()
df_d = df_d.rename(columns={'date':'날짜', '종가': '두바이유(선물)종가'})
df_d

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_11228/3005368904.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dubai['date'] = pd.to_datetime(df_dubai['date'])


,날짜,두바이유(선물)종가
0,2017-02-06,54.030
1,2017-02-13,54.220
2,2017-02-20,54.430
3,2017-02-27,54.320
4,2017-03-06,51.360
...,...,...
257,2022-01-10,82.269
258,2022-01-17,82.868
259,2022-01-24,83.455
260,2022-01-31,90.757


In [5]:
# 10년물 국채금리, 미 원유재고, 미 원유시추수 

## 미 10년물 국채 금리

In [6]:
df = pd.read_csv('./data/미10년물 국채 금리.csv')
df

,날짜,종가,오픈,고가,저가,변동 %
0,2022년 02월 10일,2.043,1.939,2.056,1.923,5.04%
1,2022년 02월 09일,1.945,1.954,1.958,1.909,-1.01%
2,2022년 02월 08일,1.965,1.921,1.970,1.921,2.37%
3,2022년 02월 07일,1.919,1.908,1.939,1.896,0.48%
4,2022년 02월 06일,1.910,1.918,1.918,1.910,-0.28%
...,...,...,...,...,...,...
1518,2017년 02월 15일,2.497,2.479,2.524,2.466,1.02%
1519,2017년 02월 14일,2.472,2.436,2.502,2.427,1.47%
1520,2017년 02월 13일,2.436,2.423,2.452,2.416,0.44%
1521,2017년 02월 12일,2.425,2.409,2.429,2.409,0.66%


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1523 entries, 0 to 1522
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      1523 non-null   object 
 1   종가      1523 non-null   float64
 2   오픈      1523 non-null   float64
 3   고가      1523 non-null   float64
 4   저가      1523 non-null   float64
 5   변동 %    1523 non-null   object 
dtypes: float64(4), object(2)
memory usage: 71.5+ KB


In [8]:
df['날짜'] = df['날짜'].str.replace('년','-')
df['날짜'] = df['날짜'].str.replace('월','-')
df['날짜'] = df['날짜'].str.replace('일','')
df['날짜'] = df['날짜'].str.replace(' ','')
df['날짜']

0       2022-02-10
1       2022-02-09
2       2022-02-08
3       2022-02-07
4       2022-02-06
           ...    
1518    2017-02-15
1519    2017-02-14
1520    2017-02-13
1521    2017-02-12
1522    2017-02-10
Name: 날짜, Length: 1523, dtype: object

In [9]:
df['날짜']= pd.to_datetime(df['날짜'])

In [10]:
df_10= df.set_index('날짜').loc['2017-2-10':'2022-02-10'].resample('W-MON',label = 'left', closed='right')['종가'].last()
df_10b = df_10.reset_index()
df_10b.columns = ['날짜','미국채10년물금리']

In [11]:
df_10b

,날짜,미국채10년물금리
0,2017-02-06,2.436
1,2017-02-13,2.436
2,2017-02-20,2.367
3,2017-02-27,2.494
4,2017-03-06,2.626
...,...,...
257,2022-01-10,1.809
258,2022-01-17,1.776
259,2022-01-24,1.784
260,2022-01-31,1.919


In [12]:
# 유가데이터와 합치기
df_du10b = pd.merge(df_10b, df_d)
df_du10b.head(3)

,날짜,미국채10년물금리,두바이유(선물)종가
0,2017-02-06,2.436,54.03
1,2017-02-13,2.436,54.22
2,2017-02-20,2.367,54.43


In [13]:
corr = df_du10b.corr()
print(corr)

            미국채10년물금리  두바이유(선물)종가
미국채10년물금리    1.000000    0.528771
두바이유(선물)종가   0.528771    1.000000


## 원유재고

In [14]:
df1 = pd.read_csv('./data/미국원유재고1.csv',delimiter='\t')
df1.head(3)

,발표일,시간,발표,예측,이전,Unnamed: 5
0,2022년 02월 17일,00:30,1.121M,-1.572M,-4.756M,NaN
1,2022년 02월 10일,00:30,-4.756M,0.369M,-1.046M,NaN
2,2022년 02월 03일,00:30,-1.046M,1.525M,2.377M,NaN


In [15]:
df1['발표일'] = df1['발표일'].str.replace('년','-')
df1['발표일'] = df1['발표일'].str.replace('월','-')
df1['발표일'] = df1['발표일'].str.replace('일','')
df1['발표일'] = df1['발표일'].str.replace(' ','')
df1['발표일'] = pd.to_datetime(df1['발표일'])
df1['발표일']

0     2022-02-17
1     2022-02-10
2     2022-02-03
3     2022-01-27
4     2022-01-21
         ...    
259   2017-03-02
260   2017-02-24
261   2017-02-16
262   2017-02-09
263   2017-02-02
Name: 발표일, Length: 264, dtype: datetime64[ns]

In [16]:
df1 = df1.rename(columns= {'발표일':'날짜','발표':'미국원유재고량'})
df1 = df1.drop(columns = ['시간','예측','이전','Unnamed: 5'])
df1.head(3)

,날짜,미국원유재고량
0,2022-02-17,1.121M
1,2022-02-10,-4.756M
2,2022-02-03,-1.046M


In [17]:
df1.head()

,날짜,미국원유재고량
0,2022-02-17,1.121M
1,2022-02-10,-4.756M
2,2022-02-03,-1.046M
3,2022-01-27,2.377M
4,2022-01-21,0.515M


In [18]:
df_oilstock= df1.set_index('날짜').loc['2017-2-10':'2022-02-10'].resample('W-MON',label = 'left', closed='right')['미국원유재고량'].last()
df_oilstock = df_oilstock.reset_index()
df_oilstock

,날짜,미국원유재고량
0,2017-02-13,9.527M
1,2017-02-20,0.564M
2,2017-02-27,1.501M
3,2017-03-06,8.209M
4,2017-03-13,-0.237M
...,...,...
256,2022-01-10,-4.553M
257,2022-01-17,0.515M
258,2022-01-24,2.377M
259,2022-01-31,-1.046M


In [19]:
df_oilstock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   날짜       261 non-null    datetime64[ns]
 1   미국원유재고량  261 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 4.2+ KB


In [20]:
# 원유재고량 float으로 바꾸기
df_oilstock['미국원유재고량'] = df_oilstock['미국원유재고량'].apply(lambda x : x.replace('M',''))
df_oilstock['미국원유재고량'] = df_oilstock['미국원유재고량'].astype(float)

In [21]:
# 합치기(dubai + oilstock duos)
df_duos = pd.merge(df_oilstock, df_d)
df_duos.head()

,날짜,미국원유재고량,두바이유(선물)종가
0,2017-02-13,9.527,54.22
1,2017-02-20,0.564,54.43
2,2017-02-27,1.501,54.32
3,2017-03-06,8.209,51.36
4,2017-03-13,-0.237,51.32


In [22]:
corr = df_duos.corr()
print(corr)  # 반비례는 맞지만 상관관계가 적다

             미국원유재고량  두바이유(선물)종가
미국원유재고량     1.000000   -0.101909
두바이유(선물)종가 -0.101909    1.000000


## 미국 베이커휴즈 총 원유시추수

In [24]:
pwd

'C:\\KDT4\\fintech\\프로젝트1\\final코드'

In [26]:
df2 = pd.read_csv('./data/미국베이커휴즈원유시추수.csv',sep='\t')  # 2018년 1월 6일 ~ 2022년 2월 
df2.head(3)

,발표일,시간,발표,예측,이전,Unnamed: 5
0,2022년 02월 19일,03:00,,,613,NaN
1,2022년 02월 12일,03:00,635,,613,NaN
2,2022년 02월 05일,03:00,613,,610,NaN


In [27]:
df2.tail(3)

,발표일,시간,발표,예측,이전,Unnamed: 5
211,2018년 01월 20일,03:00,936.0,,939.0,NaN
212,2018년 01월 13일,03:00,939.0,,924.0,NaN
213,2018년 01월 06일,03:00,924.0,,929.0,NaN


In [28]:
df2.drop(columns=['시간','예측','이전','Unnamed: 5'], inplace= True)
df2.head(3)

,발표일,발표
0,2022년 02월 19일,
1,2022년 02월 12일,635
2,2022년 02월 05일,613


In [29]:
# 결측치 확인
df2.isnull().sum()

발표일    0
발표     0
dtype: int64

In [30]:
# 0행 1행 삭제 (유가데이터 2월 7일까지이므로)
df2 = df2.drop(index=[0,1])
df2.head(2)

,발표일,발표
2,2022년 02월 05일,613
3,2022년 01월 29일,610


In [31]:
df2['발표일'] = df2['발표일'].str.replace('년','-')
df2['발표일'] = df2['발표일'].str.replace('월','-')
df2['발표일'] = df2['발표일'].str.replace('일','')
df2['발표일'] = df2['발표일'].str.replace(' ','')
df2['발표일'] = pd.to_datetime(df2['발표일'])

In [32]:
df2.head(3)

,발표일,발표
2,2022-02-05,613
3,2022-01-29,610
4,2022-01-22,604


In [33]:
df2.columns = ['날짜', '원유시추수']
df2.head(3)

,날짜,원유시추수
2,2022-02-05,613
3,2022-01-29,610
4,2022-01-22,604


In [34]:
df2['날짜'] = df2['날짜'] + timedelta(days =2)
df2['날짜']

2     2022-02-07
3     2022-01-31
4     2022-01-24
5     2022-01-17
6     2022-01-10
         ...    
209   2018-02-05
210   2018-01-29
211   2018-01-22
212   2018-01-15
213   2018-01-08
Name: 날짜, Length: 212, dtype: datetime64[ns]

In [35]:
# 유가데이터 맞춰주기

df_db = df_d.set_index('날짜').loc['2018-01-06':]
df_db = df_db.reset_index()
df_db


,날짜,두바이유(선물)종가
0,2018-01-08,66.360
1,2018-01-15,65.970
2,2018-01-22,66.230
3,2018-01-29,64.990
4,2018-02-05,61.160
...,...,...
209,2022-01-10,82.269
210,2022-01-17,82.868
211,2022-01-24,83.455
212,2022-01-31,90.757


In [36]:
# 유가데이터 원유시추 합치기

df_dubh = pd.merge(df2, df_db, on='날짜')
df_dubh

,날짜,원유시추수,두바이유(선물)종가
0,2022-02-07,613,89.883
1,2022-01-31,610,90.757
2,2022-01-24,604,83.455
3,2022-01-17,601,82.868
4,2022-01-10,588,82.269
...,...,...,...
194,2018-02-05,946.0,61.160
195,2018-01-29,947.0,64.990
196,2018-01-22,936.0,66.230
197,2018-01-15,939.0,65.970


In [37]:
df_dubh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          199 non-null    datetime64[ns]
 1   원유시추수       199 non-null    object        
 2   두바이유(선물)종가  193 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 6.2+ KB


In [38]:
df_dubh['원유시추수'] = df_dubh['원유시추수'].apply(lambda x :x.replace(',',''))

In [39]:
df_dubh['원유시추수'] = df_dubh['원유시추수'].astype(float)

In [40]:
df_dubh

,날짜,원유시추수,두바이유(선물)종가
0,2022-02-07,613.0,89.883
1,2022-01-31,610.0,90.757
2,2022-01-24,604.0,83.455
3,2022-01-17,601.0,82.868
4,2022-01-10,588.0,82.269
...,...,...,...
194,2018-02-05,946.0,61.160
195,2018-01-29,947.0,64.990
196,2018-01-22,936.0,66.230
197,2018-01-15,939.0,65.970


In [41]:
corr = df_dubh.corr()
print(corr)

               원유시추수  두바이유(선물)종가
원유시추수       1.000000    0.411272
두바이유(선물)종가  0.411272    1.000000


In [55]:
df_dubh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   날짜          199 non-null    datetime64[ns]
 1   원유시추수       199 non-null    float64       
 2   두바이유(선물)종가  193 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 6.2 KB


In [64]:
df_dubh
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df_dubh.set_index('날짜')), columns = ['원유시추수', '두바이유(선물)종가'])
df

,원유시추수,두바이유(선물)종가
0,0.439809,0.987582
1,0.436234,1.000000
2,0.429082,0.896253
3,0.425507,0.887913
4,0.410012,0.879403
...,...,...
194,0.836710,0.579487
195,0.837902,0.633903
196,0.824791,0.651521
197,0.828367,0.647827


In [63]:
#시각화

df_dubh
px.line(df.reset_index(), x= 'index', y=['원유시추수', '두바이유(선물)종가'])

### opec원유생산량

In [74]:
# 사우디
df_s = pd.read_csv('./data/사우디원유생산량.csv',delimiter='\t',usecols=['발표일','발표'])
df_s.columns = ['날짜', '사우디원유생산량']
df_s['날짜'] = df_s['날짜'].str.replace('년','-')
df_s['날짜'] = df_s['날짜'].str.replace('월','-')
df_s['날짜'] = df_s['날짜'].str.replace('일','')
df_s['날짜'] = df_s['날짜'].str.replace(' ','')
df_s['날짜'] = pd.to_datetime(df_s['날짜'])
df_s['사우디원유생산량'] = df_s['사우디원유생산량'].apply(lambda x : x.replace('M',''))
df_s['사우디원유생산량'] = df_s['사우디원유생산량'].astype(float)
df_s.head()



,날짜,사우디원유생산량
0,2022-02-01,9.93
1,2021-12-31,9.87
2,2021-12-01,9.76
3,2021-10-30,9.68
4,2021-10-01,9.49


In [75]:
# 이라크
df_i = pd.read_csv('./data/이라크원유생산량.csv',delimiter='\t',usecols=['발표일','발표'])
df_i.columns = ['날짜', '이라크원유생산량']
df_i['날짜'] = df_i['날짜'].str.replace('년','-')
df_i['날짜'] = df_i['날짜'].str.replace('월','-')
df_i['날짜'] = df_i['날짜'].str.replace('일','')
df_i['날짜'] = df_i['날짜'].str.replace(' ','')
df_i['날짜'] = pd.to_datetime(df_i['날짜'])
df_i['이라크원유생산량'] = df_i['이라크원유생산량'].apply(lambda x : x.replace('M',''))
df_i['이라크원유생산량'] = df_i['이라크원유생산량'].astype(float)
df_i.head()

,날짜,이라크원유생산량
0,2022-02-01,4.27
1,2021-12-31,4.24
2,2021-12-01,4.15
3,2021-10-30,4.14
4,2021-10-01,4.06


In [89]:
# 아랍에미리트
df_a = pd.read_csv('./data/아랍에미리트원유생산량.csv',delimiter='\t',usecols=['발표일','발표'])
df_a.columns = ['날짜', '아랍에미리트원유생산량']
df_a['날짜'] = df_a['날짜'].str.replace('년','-')
df_a['날짜'] = df_a['날짜'].str.replace('월','-')
df_a['날짜'] = df_a['날짜'].str.replace('일','')
df_a['날짜'] = df_a['날짜'].str.replace(' ','')
df_a['날짜'] = pd.to_datetime(df_a['날짜'])
# df_a['아랍에미리트원유생산량']
df_a['아랍에미리트원유생산량'] = df_a['아랍에미리트원유생산량'].apply(lambda x : x.replace('M',''))
# 빈값 처리
df_a['아랍에미리트원유생산량'][24] = np.nan

df_a['아랍에미리트원유생산량'] = df_a['아랍에미리트원유생산량'].astype(float)

#결측치 평균값으로 처리
df_a['아랍에미리트원유생산량'] = df_a['아랍에미리트원유생산량'].interpolate()
df_a

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_11228/2559541683.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,날짜,아랍에미리트원유생산량
0,2022-02-01,2.88
1,2021-12-31,2.85
2,2021-12-01,2.83
3,2021-10-30,2.79
4,2021-10-01,2.78
5,2021-09-01,2.72
6,2021-07-31,2.68
7,2021-07-01,2.64
8,2021-06-01,2.61
9,2021-05-01,2.61


In [93]:
# opec 상위3개국가 생산량 합

df_merge = pd.merge(df_s,df_i)
df_opec = pd.merge(df_merge,df_a)
df_opec['opec3개국생산량'] = df_opec.sum(axis=1)
df_opec

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_11228/48815423.py:5: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,날짜,사우디원유생산량,이라크원유생산량,아랍에미리트원유생산량,opec3개국생산량
0,2022-02-01,9.93,4.27,2.88,17.08
1,2021-12-31,9.87,4.24,2.85,16.96
2,2021-12-01,9.76,4.15,2.83,16.74
3,2021-10-30,9.68,4.14,2.79,16.61
4,2021-10-01,9.49,4.06,2.78,16.33
5,2021-09-01,9.40,3.98,2.72,16.10
6,2021-07-31,8.91,3.94,2.68,15.53
7,2021-07-01,8.47,3.97,2.64,15.08
8,2021-06-01,8.13,3.92,2.61,14.66
9,2021-05-01,8.09,3.91,2.61,14.61


In [94]:
# 유가데이터 날짜 맞춰주기

In [104]:
df = pd.read_csv('./data/daum_oil.csv')
df_dubai = df[df['company'] == '두바이유선물']
df_dubai['date'] = pd.to_datetime(df_dubai['date'])
df_d1 = df_dubai.set_index('date').loc['2019-9':'2022-02'].resample('M',label = 'left', closed='right')['종가'].last()
# df_d1 = df_d1.reset_index()
# df_d1 = df_d1.rename(columns={'date':'날짜', '종가': '두바이유(선물)종가'})
# df_d1
df_total = pd.merge(left= df_opec ,right=df_dubai, how='inner', left_on = '날짜', right_on = 'date')
df_total

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_11228/3595781068.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,날짜,사우디원유생산량,이라크원유생산량,아랍에미리트원유생산량,opec3개국생산량,company,date,종가,전일비,등락률
0,2022-02-01,9.93,4.27,2.88,17.08,두바이유선물,2022-02-01,83.455,0.000,0.00
1,2021-12-31,9.87,4.24,2.85,16.96,두바이유선물,2021-12-31,73.185,-0.020,-0.03
2,2021-12-01,9.76,4.15,2.83,16.74,두바이유선물,2021-12-01,80.276,0.000,0.00
3,2021-10-01,9.49,4.06,2.78,16.33,두바이유선물,2021-10-01,72.608,0.000,0.00
4,2021-09-01,9.40,3.98,2.72,16.10,두바이유선물,2021-09-01,69.485,0.000,0.00
5,2021-07-01,8.47,3.97,2.64,15.08,두바이유선물,2021-07-01,71.573,0.000,0.00
6,2021-06-01,8.13,3.92,2.61,14.66,두바이유선물,2021-06-01,66.275,0.000,0.00
7,2021-04-01,8.12,3.89,2.61,14.62,두바이유선물,2021-04-01,64.414,0.000,0.00
8,2020-12-01,8.96,3.84,2.44,15.24,두바이유선물,2020-12-01,43.394,-0.126,-0.29
9,2020-10-01,8.89,3.65,2.71,15.25,두바이유선물,2020-10-01,41.495,0.000,0.00
